In [ ]:
from accelerate.utils import write_basic_config

# write_basic_config()

In [ ]:
!pip install imagen-pytorch

In [ ]:
!conda install -c defaults intel-openmp -f

In [ ]:
!conda install -c pytorch pytorch

In [ ]:
import torch
from imagen_pytorch import Unet, Imagen, SRUnet256, ImagenTrainer

# unets for unconditional imagen

unet1 = Unet(
    dim = 32,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True),
    layer_cross_attns = False,
    use_linear_attn = True
)

unet2 = SRUnet256(
    dim = 32,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = (2, 4, 8),
    layer_attns = (False, False, True),
    layer_cross_attns = False
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    condition_on_text = False,   # this must be set to False for unconditional Imagen
    unets = (unet1, unet2),
    image_sizes = (64, 128),
    timesteps = 1000
)

trainer = ImagenTrainer(imagen)

# now get a ton of images and feed it through the Imagen trainer

training_images = torch.randn(13800, 3, 256, 256)

# train each unet separately
# in this example, only training on unet number 1

loss = trainer(training_images, unet_number = 1)
trainer.update(unet_number = 1)

# do the above for many many many many steps
# now you can sample images unconditionally from the cascading unet(s)

images = trainer.sample(batch_size = 16) # (16, 3, 128, 128)

#### Using Data-loader to supply my own images

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os

# Specify the path to your image folder
image_folder = "firstDataSetRun/"

# Define a transformation to preprocess the images
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize images to the required dimensions
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize pixel values
])

# List to store processed images
processed_images = []
i=0
# Loop through each image in the folder
for filename in os.listdir(image_folder):
    # Load the image
    img_path = os.path.join(image_folder, filename)
    img = Image.open(img_path).convert("RGB")

    # Apply the preprocessing transformations
    img_tensor = preprocess(img)

    # Append the processed image to the list
    processed_images.append(img_tensor)
    print(f"{i} done")
    i+=1

# Stack the processed images into a single tensor
training_images = torch.stack(processed_images)

# Print the shape of the resulting tensor
print(training_images.shape)


In [ ]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer

# unets for unconditional imagen

unet1 = Unet(
    dim = 32,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True),
    layer_cross_attns = False,
    use_linear_attn = True
)

unet2 = SRUnet256(
    dim = 32,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = (2, 4, 8),
    layer_attns = (False, False, True),
    layer_cross_attns = False
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    condition_on_text = False,   # this must be set to False for unconditional Imagen
    unets = (unet1, unet2),
    image_sizes = (64, 128),
    timesteps = 1000
)

trainer = ImagenTrainer(imagen)

# now get a ton of images and feed it through the Imagen trainer

# training_images = torch.randn(4, 3, 256, 256)

# train each unet separately
# in this example, only training on unet number 1

loss = trainer(training_images, unet_number = 1)
trainer.update(unet_number = 1)

# do the above for many many many many steps
# now you can sample images unconditionally from the cascading unet(s)

images = trainer.sample(batch_size = 16) # (16, 3, 128, 128)

In [ ]:
!pip install tensorflow

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


class CustomImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

# Example usage:
folder_path = "firstDataSetRun"
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = CustomImageDataset("firstDataSetRun", transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Inspect the dataset by visualizing a few images
def show_images(images):
    fig, axs = plt.subplots(1, len(images), figsize=(12, 4))
    for i, img in enumerate(images):
        img_np = np.transpose(img.numpy(), (1, 2, 0))
        axs[i].imshow(img_np)
        axs[i].axis('off')
        image_tensor = tf.convert_to_tensor(img)
        # Get the dimensions of the image
        print(image_tensor.shape)

    plt.show()

# Visualize a few batches of images
for batch in dataloader:
    show_images(batch)
    break  # Break after the first batch to keep the example concise


In [ ]:
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import time

# unets for unconditional imagen
class CustomImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image

# Example usage:
# folder_path = "firstDataSetRun"
# transform = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
# ])

#####################################      FIRST TRY       #####################################
# unet = Unet(
#     dim = 32,
#     dim_mults = (1, 2, 4, 8),
#     num_resnet_blocks = 1,
#     layer_attns = (False, False, False, True),
#     layer_cross_attns = False
# )

#####################################     SECOND TRY WITH BIGGER MODEL      #####################################

# unet = Unet(
#     dim = 512,
#     dim_mults = (1, 2, 4, 8),
#     num_resnet_blocks= (2, 4, 8, 8),
#     layer_attns = (False, True, True, True),
#     layer_cross_attns = False,
#     attn_heads= 8,
#     memory_efficient= True
# )

# imagen = Imagen(
#     condition_on_text = False,  # this must be set to False for unconditional Imagen
#     unets = unet,
#     image_sizes = 256,
#     timesteps = 1000
# )


#####################################     THIRD TRY WITH BIGGER MODEL & 2 U-NETS      #####################################

# unet1 = Unet(
#     dim = 512,
#     dim_mults = (1, 2, 4, 8),
#     num_resnet_blocks: (2, 4, 8, 8)
#     layer_attns = (False, True, True, True),
#     layer_cross_attns = False,
#     attn_heads: 8,
#     memory_efficient: True
# )

# unet2 = BaseUnet64

# imagen = Imagen(
#     condition_on_text = False,  # this must be set to False for unconditional Imagen
#     unets = (unet1, unet2),
#     image_sizes = 256,
#     timesteps = 1000
# )
#####################################     THIRD TRY WITH MODEL ARC. FROM PAPER      #####################################

unet1 = Unet(
    dim=32,
    dim_mults=(1, 2, 4),
    num_resnet_blocks=3,
    layer_attns=(False, True, True),
    layer_cross_attns=(False, True, True),
    use_linear_attn=True,
)

unet2 = Unet(
        dim=64,
        dim_mults=(1, 2, 4),
        num_resnet_blocks=3,
        layer_attns=(False, True, True),
        layer_cross_attns=(False, True, True),
        use_linear_attn=True,
)

imagen = Imagen(
    condition_on_text = False,  # this must be set to False for unconditional Imagen
    unets = [unet1, unet2],
    image_sizes = 64,
    timesteps = 1000
)
###############################################################################################################

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
)

# instantiate your dataloader, which returns the necessary inputs to the DDPM as tuple in the order of images, text embeddings, then text masks. in this case, only images is returned as it is unconditional training


# working training loop

total_epochs = 10


print("Going into training loop")

if __name__ == "__main__":
    data_folder = sys.argv[1]
    batch_size_training=sys.argv[2]
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    dataset = Dataset(data_folder, image_size = 64)
    trainer.add_train_dataset(dataset, batch_size = batch_size_training)

    np.random.seed(2018)

    start_time_main=time.time()

    for i in range(250000):
        loss = trainer.train_step(unet_number = 1, max_batch_size = 4)
        print(f'loss: {loss}')
        
        if not (i % 50):
            valid_loss = trainer.valid_step(unet_number = 1, max_batch_size = 4)
            print(f'valid loss: {valid_loss}')
    
        if not (i % 100) and trainer.is_main: # is_main makes sure this can run in distributed
            images = trainer.sample(batch_size = 1, return_pil_images = True) # returns List[Image]
            images[0].save(f'./sample-{i // 100}.png')
            trainer.save('largerModelWithCorrectData.pt')
    stop_time_main = time.time()
    elapsed_time = end_time - start_time
    print(f"Execution time: {elapsed_time}")
    
    
    # for epoch in range(total_epochs):
    #     print(f"epoch {epoch}")
    #     for batch_data in dataloader:
    #         print(f"batch")
    #         loss = trainer.train_step(unet_number=1, max_batch_size=batch_size_training)
    #         print(f'Epoch {epoch + 1}, Loss: {loss}')
    
    #     # Validation at the end of each epoch
    #     valid_loss = trainer.valid_step(unet_number=1, max_batch_size=batch_size_training)
    #     print(f'Validation Loss: {valid_loss}')
    
    #     if trainer.is_main:
    #         # Image sampling every 2 epochs
    #         if epoch % 2 == 0:
    #             images = trainer.sample(batch_size=1, return_pil_images=True)
    #             images[0].save(f'./sample-{epoch // 2}.png')
    #     epoch+=1



In [ ]:
# for i in range(200000):
#     loss = trainer.train_step(unet_number = 1, max_batch_size = 4)
#     print(f'loss: {loss}')

#     if not (i % 50):
#         valid_loss = trainer.valid_step(unet_number = 1, max_batch_size = 4)
#         print(f'valid loss: {valid_loss}')

#     if not (i % 100) and trainer.is_main: # is_main makes sure this can run in distributed
#         images = trainer.sample(batch_size = 1, return_pil_images = True) # returns List[Image]
#         images[0].save(f'./sample-{i // 100}.png')

### BASE3.PY Troubleshooting

In [ ]:

from imagen_pytorch import Unet, Imagen, ImagenTrainer, BaseUnet64
from imagen_pytorch.data import Dataset
# from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import time

# unets for unconditional imagen
class CustomImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image


#####################################     THIRD TRY WITH BIGGER MODEL & 2 U-NETS      #####################################
unet1 = Unet(
    dim=32,
    dim_mults=(1, 2, 4),
    num_resnet_blocks=3,
    layer_attns=(False, True, True),
    layer_cross_attns=(False, True, True),
    use_linear_attn=True,
)

unet2 = BaseUnet64()

imagen = Imagen(
    condition_on_text = False,  # this must be set to False for unconditional Imagen
    unets = [unet1, unet2],
    # channels=1,
    image_sizes=(32, 64),
    timesteps = 1000
)


trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
)

# instantiate your dataloader, which returns the necessary inputs to the DDPM as tuple in the order of images, text embeddings, then text masks. in this case, only images is returned as it is unconditional training


# working training loop

total_epochs = 10


print("Going into training loop")

if __name__ == "__main__":
    data_folder = "firstDataSetRun" # sys.argv[0]
    batch_size_training= 16 #sys.argv[1]
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
    ])
    dataset = Dataset(data_folder, image_size = 64)
    trainer.add_train_dataset(dataset, batch_size = batch_size_training)

    np.random.seed(2018)

    start_time_main=time.time()

    # for i in range(300000):
    #     for u in range(1,2):
    #         loss = trainer(training_images, unet_number=u, max_batch_size=max_batch_size)
    #         trainer.update(unet_number=u)
    
    # images = trainer.sample(batch_size=16, return_pil_images=True)
    # trainer.save('largerModelWithCorrectData.pt')
    # stop_time_main = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Execution time: {elapsed_time}")

    for i in range(200000):
        for u in range(1, 3):  # Loop over both U-Nets
            # Training step
            loss = trainer.train_step(unet_number=u, max_batch_size=4)
            print(f'Training loss at step {i} for U-Net {u}: {loss}')
    
            # Validation step every 50 steps
            if i % 50 == 0:
                valid_loss = trainer.valid_step(unet_number=u, max_batch_size=4)
                print(f'Validation loss at step {i} for U-Net {u}: {valid_loss}')
    
            # Save a sample image every 100 steps
            if i % 100 == 0 and trainer.is_main:  # is_main ensures this can run in distributed mode
                images = trainer.sample(batch_size=1, unet_number=u, return_pil_images=True)  # returns List[Image]
                images[0].save(f'./sample-{u}-{i // 100}.png')



    
    # for i in range(300000):
    #     for u in range(1,2)
    #     loss = trainer.train_step(unet_number = u, max_batch_size = 4)
    #     print(f'loss: {loss}')
        
    #     if not (i % 50):
    #         valid_loss = trainer.valid_step(unet_number = u, max_batch_size = 4)
    #         print(f'valid loss: {valid_loss}')
    
    #     if not (i % 100) and trainer.is_main: # is_main makes sure this can run in distributed
    #         images = trainer.sample(batch_size = 1, return_pil_images = True) # returns List[Image]
    #         images[0].save(f'./sample-{i // 100}.png')
    #         trainer.save('largerModelWithCorrectData.pt')
    # stop_time_main = time.time()
    # elapsed_time = end_time - start_time
    # print(f"Execution time: {elapsed_time}")



In [ ]:
 for i in range(200000):
    # Training step
    loss = trainer.train_step(only_train_unet_number=1, max_batch_size=4)
    print(f'Training loss at step {i} for U-Net 1: {loss}')

    # Validation step every 50 steps
    if i % 50 == 0:
        valid_loss = trainer.valid_step(unet_number=1, max_batch_size=4)
        print(f'Validation loss at step {i} for U-Net 1: {valid_loss}')

 for i in range(200000):
    # Training step
    loss = trainer.train_step(only_train_unet_number=2, max_batch_size=4)
    print(f'Training loss at step {i} for U-Net 2: {loss}')

    # Validation step every 50 steps
    if i % 50 == 0:
        valid_loss = trainer.valid_step(unet_number=2, max_batch_size=4)
        print(f'Validation loss at step {i} for U-Net 2: {valid_loss}')

In [ ]:
import torch
from imagen_pytorch import Unet, Imagen, SRUnet256, ImagenTrainer

# unets for unconditional imagen

unet1 = Unet(
    dim = 128,
    dim_mults = (1, 2, 4),
    num_resnet_blocks = 0,
    layer_attns = (False, True, True),
    layer_cross_attns = False,
    use_linear_attn = True
)


# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    condition_on_text = False,   # this must be set to False for unconditional Imagen
    unets = (unet1),
    image_sizes = (256),
    timesteps = 1000
)